In [ ]:
!pip install autokeras


     |████████████████████████████████| 162 kB 19.7 MB/s 
     |████████████████████████████████| 133 kB 44.2 MB/s 
     |████████████████████████████████| 462 kB 60.8 MB/s 


In [ ]:
import os

import numpy as np
import tensorflow as tf
from sklearn.datasets import load_files

import autokeras as ak


## A Simple Example
The first step is to prepare your data. Here we use the [IMDB
dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification)
as an example.


In [ ]:

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz",
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract=True,
)

# set path to dataset
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), "aclImdb")

classes = ["pos", "neg"]
train_data = load_files(
    os.path.join(IMDB_DATADIR, "train"), shuffle=True, categories=classes
)
test_data = load_files(
    os.path.join(IMDB_DATADIR, "test"), shuffle=False, categories=classes
)

x_train = np.array(train_data.data)
y_train = np.array(train_data.target)
x_test = np.array(test_data.data)
y_test = np.array(test_data.target)

print(x_train.shape)  # (25000,)
print(y_train.shape)  # (25000, 1)
print(x_train[0][:50])  # this film was just brilliant casting


84140032/84125825 [==============================] - 5s 0us/step
(25000,)
(25000,)
b'Zero Day leads you to think, even re-think why two'


The second step is to run the [TextClassifier](/text_classifier).  As a quick
demo, we set epochs to 2.  You can also leave the epochs unspecified for an
adaptive number of epochs.


In [ ]:

# Initialize the text classifier.
clf = ak.TextClassifier(
    overwrite=True, max_trials=1
)  # It only tries 1 model as a quick demo.
# Feed the text classifier with training data.
clf.fit(x_train, y_train, epochs=2)
# Predict with the best model.
predicted_y = clf.predict(x_test)
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))



Trial 1 Complete [00h 04m 34s]
val_loss: 0.2799142599105835

Best val_loss So Far: 0.2799142599105835
Total elapsed time: 00h 04m 34s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
782/782 [==============================] - 150s 190ms/step - loss: 0.4392 - accuracy: 0.7722
Epoch 2/2
782/782 [==============================] - 148s 190ms/step - loss: 0.2411 - accuracy: 0.9024
INFO:tensorflow:Assets written to: ./text_classifier/best_model/assets
782/782 [==============================] - 39s 50ms/step - loss: 0.2657 - accuracy: 0.8917
[0.265703946352005, 0.8916800022125244]


## Validation Data
By default, AutoKeras use the last 20% of training data as validation data.  As
shown in the example below, you can use `validation_split` to specify the
percentage.


In [ ]:
clf.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
)


You can also use your own validation set instead of splitting it from the
training data with `validation_data`.


In [ ]:
split = 5000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
clf.fit(
    x_train,
    y_train,
    epochs=2,
    # Use your own validation set.
    validation_data=(x_val, y_val),
)


## Customized Search Space
For advanced users, you may customize your search space by using
[AutoModel](/auto_model/#automodel-class) instead of
[TextClassifier](/text_classifier). You can configure the
[TextBlock](/block/#textblock-class) for some high-level configurations, e.g.,
`vectorizer` for the type of text vectorization method to use.  You can use
'sequence', which uses [TextToInteSequence](/block/#texttointsequence-class) to
convert the words to integers and use [Embedding](/block/#embedding-class) for
embedding the integer sequences, or you can use 'ngram', which uses
[TextToNgramVector](/block/#texttongramvector-class) to vectorize the
sentences.  You can also do not specify these arguments, which would leave the
different choices to be tuned automatically.  See the following example for
detail.


In [ ]:

input_node = ak.TextInput()
output_node = ak.TextBlock(block_type="ngram")(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=2)


Trial 1 Complete [00h 00m 37s]
val_loss: 0.36526966094970703

Best val_loss So Far: 0.36526966094970703
Total elapsed time: 00h 00m 37s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
157/157 [==============================] - 12s 74ms/step - loss: 0.4502 - accuracy: 0.7932
Epoch 2/2
157/157 [==============================] - 12s 75ms/step - loss: 0.0908 - accuracy: 0.9798
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


The usage of [AutoModel](/auto_model/#automodel-class) is similar to the
[functional API](https://www.tensorflow.org/guide/keras/functional) of Keras.
Basically, you are building a graph, whose edges are blocks and the nodes are
intermediate outputs of blocks.  To add an edge from `input_node` to
`output_node` with `output_node = ak.[some_block]([block_args])(input_node)`.

You can even also use more fine grained blocks to customize the search space
even further. See the following example.


In [ ]:

input_node = ak.TextInput()
output_node = ak.TextToIntSequence()(input_node)
output_node = ak.Embedding()(output_node)
# Use separable Conv layers in Keras.
output_node = ak.ConvBlock(separable=True)(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=2)


Trial 1 Complete [00h 00m 22s]
val_loss: 0.6931468844413757

Best val_loss So Far: 0.6931468844413757
Total elapsed time: 00h 00m 22s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
157/157 [==============================] - 9s 53ms/step - loss: 0.6932 - accuracy: 0.4876
Epoch 2/2
157/157 [==============================] - 8s 52ms/step - loss: 0.6932 - accuracy: 0.4928
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


## Data Format
The AutoKeras TextClassifier is quite flexible for the data format.

For the text, the input data should be one-dimensional For the classification
labels, AutoKeras accepts both plain labels, i.e. strings or integers, and
one-hot encoded encoded labels, i.e. vectors of 0s and 1s.

We also support using [tf.data.Dataset](
https://www.tensorflow.org/api_docs/python/tf/data/Dataset?version=stable)
format for the training data.


In [ ]:
train_set = tf.data.Dataset.from_tensor_slices(((x_train,), (y_train,))).batch(32)
test_set = tf.data.Dataset.from_tensor_slices(((x_test,), (y_test,))).batch(32)

clf = ak.TextClassifier(overwrite=True, max_trials=2)
# Feed the tensorflow Dataset to the classifier.
clf.fit(train_set, epochs=2)
# Predict with the best model.
predicted_y = clf.predict(test_set)
# Evaluate the best model with testing data.
print(clf.evaluate(test_set))


Trial 2 Complete [00h 00m 44s]
val_loss: 0.3801417350769043

Best val_loss So Far: 0.3801417350769043
Total elapsed time: 00h 01m 41s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
157/157 [==============================] - 18s 98ms/step - loss: 0.6892 - accuracy: 0.5334
Epoch 2/2
157/157 [==============================] - 15s 98ms/step - loss: 0.5037 - accuracy: 0.7586
INFO:tensorflow:Assets written to: ./text_classifier/best_model/assets
782/782 [==============================] - 24s 30ms/step - loss: 0.4184 - accuracy: 0.8075
[0.41844579577445984, 0.807479977607727]


## Reference
[TextClassifier](/text_classifier),
[AutoModel](/auto_model/#automodel-class),
[TextBlock](/block/#textblock-class),
[TextToInteSequence](/block/#texttointsequence-class),
[Embedding](/block/#embedding-class),
[TextToNgramVector](/block/#texttongramvector-class),
[ConvBlock](/block/#convblock-class),
[TextInput](/node/#textinput-class),
[ClassificationHead](/block/#classificationhead-class).
